In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split




normalization done in conjunction with paper under supplementary materials section

In [2]:
dataset_path = 'W2S_raw/raw'  
save_path = 'normalized_images'  

# Function to process and save each normalized image individually
def process_and_save_images(image_paths, mean_value, std_value, image_type):
    os.makedirs(save_path, exist_ok=True)  
   
    for i, image_path in enumerate(tqdm(image_paths, desc=f'Processing {image_type} images')):
        
        image = np.load(image_path)
        normalized_image = (image - mean_value) / std_value
        filename = os.path.basename(image_path)
        np.save(os.path.join(save_path, f'normalized_{image_type}_{filename}'), normalized_image)

# Function to calculate mean and standard deviation from a set of image paths
def calculate_mean_std(image_paths):
    pixel_sum = 0
    pixel_sum_squared = 0
    pixel_count = 0

    for image_path in tqdm(image_paths, desc='Calculating mean and std'):
        image = np.load(image_path)
        pixel_sum += np.sum(image)
        pixel_sum_squared += np.sum(image ** 2)
        pixel_count += image.size

    mean_value = pixel_sum / pixel_count
    std_value = np.sqrt(pixel_sum_squared / pixel_count - mean_value ** 2)

    return mean_value, std_value


In [3]:

# Find all the LR and HR image paths
lr_image_paths = glob.glob(os.path.join(dataset_path, '**', 'wf_channel*.npy'), recursive=True)
hr_image_paths = glob.glob(os.path.join(dataset_path, '**', 'sim_channel*.npy'), recursive=True)

# Check that we found both LR and HR images
if not lr_image_paths or not hr_image_paths:
    raise ValueError(f"No images found with pattern 'wf_channel*.npy' or 'sim_channel*.npy' in the directory {dataset_path}")


In [ ]:

# Calculate mean and std for LR images
mean_value_lr, std_value_lr = calculate_mean_std(lr_image_paths)

# Process and save normalized LR images
process_and_save_images(lr_image_paths, mean_value_lr, std_value_lr, image_type='lr')

# Calculate mean and std for HR images
mean_value_hr, std_value_hr = calculate_mean_std(hr_image_paths)

process_and_save_images(hr_image_paths, mean_value_hr, std_value_hr, image_type='hr')

In [5]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        # Encoder path
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 512))

        # Decoder path
        self.up1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv1 = DoubleConv(512, 256)  # Adjusted
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv2 = DoubleConv(256, 128)  # Adjusted
        self.up3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv3 = DoubleConv(128, 64)  # Adjusted
        self.up4 = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2)
        self.conv4 = DoubleConv(64, 64)

        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.inc(x)  # Shape: (B, 64, H, W)
        x2 = self.down1(x1)  # Shape: (B, 128, H//2, W//2)
        x3 = self.down2(x2)  # Shape: (B, 256, H//4, W//4)
        x4 = self.down3(x3)  # Shape: (B, 512, H//8, W//8)
        x5 = self.down4(x4)  # Shape: (B, 512, H//16, W//16)

        # Decoder
        x = self.up1(x5)  # Upsampling to match x3
        if x.shape[2:] != x3.shape[2:]:
            # Adjust x spatial dimensions to match x3
            diff = [x3.shape[2] - x.shape[2], x3.shape[3] - x.shape[3]]
            x = nn.functional.pad(x, [diff[1] // 2, diff[1] - diff[1] // 2, diff[0] // 2, diff[0] - diff[0] // 2])

        # Concatenation and further processing
        x = torch.cat([x, x3], dim=1)  # Concatenating after padding if necessary
        x = self.conv1(x)

        x = self.up2(x)  # Upsampling to match x2
        if x.shape[2:] != x2.shape[2:]:
            diff = [x2.shape[2] - x.shape[2], x2.shape[3] - x.shape[3]]
            x = nn.functional.pad(x, [diff[1] // 2, diff[1] - diff[1] // 2, diff[0] // 2, diff[0] - diff[0] // 2])
        
        x = torch.cat([x, x2], dim=1)  # Concatenating after padding if necessary
        x = self.conv2(x)

        x = self.up3(x)  # Upsampling to match x1
        if x.shape[2:] != x1.shape[2:]:
            diff = [x1.shape[2] - x.shape[2], x1.shape[3] - x.shape[3]]
            x = nn.functional.pad(x, [diff[1] // 2, diff[1] - diff[1] // 2, diff[0] // 2, diff[0] - diff[0] // 2])
        
        x = torch.cat([x, x1], dim=1)  # Concatenating after padding if necessary
        x = self.conv3(x)

        x = self.up4(x)  # Final upsampling
        x = self.conv4(x)  # Final convolutions
        
        logits = self.outc(x)  # Final output
        
        return logits


In [6]:
class NumpyDataset(Dataset):
    def __init__(self, lr_images, hr_images):
        self.lr_images = lr_images
        self.hr_images = hr_images

    def __len__(self):
        return len(self.lr_images)

    def __getitem__(self, idx):
        lr_image = np.load(self.lr_images[idx]).astype(np.float32)
        hr_image = np.load(self.hr_images[idx]).astype(np.float32)
       
        if lr_image.shape[0] == 400:  
            lr_image = lr_image[0]  # first slice 

        
        hr_image = np.expand_dims(hr_image, axis=0)  
        lr_image = np.expand_dims(lr_image, axis=0)

        return torch.from_numpy(lr_image), torch.from_numpy(hr_image)



there are NaN's in the data

In [7]:
dataset = NumpyDataset(lr_image_paths, hr_image_paths)

train_proportion = 0.8

# Calculate the number of samples for train and validation
total_images = len(dataset)
train_count = int(total_images * train_proportion)
valid_count = total_images - train_count

# Split the dataset
train_dataset, valid_dataset = random_split(dataset, [train_count, valid_count])

# Smaller batch size to reduce memory usage
batch_size = 2  # Adjust as needed

# DataLoader with smaller batch size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Model, loss, and optimizer initialization
model = UNet(n_channels=1, n_classes=1)
if torch.cuda.is_available():
    model.cuda()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Gradient accumulation steps for reduced memory usage
accumulation_steps = 4  

# Training loop with gradient accumulation
n_epochs = 100
for epoch in range(n_epochs):
    model.train()  # Ensure the model is in training mode
    optimizer.zero_grad()  # Reset gradients
    train_loss = 0  # Initialize train_loss to track loss accumulation for the epoch

    for i, (lr_images, hr_images) in enumerate(tqdm(train_dataloader, desc='Training')):
        # Convert to CUDA tensors if needed
        if torch.cuda.is_available():
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()

        outputs = model(lr_images)
        loss = criterion(outputs, hr_images)
        loss.backward()  # Accumulate gradients

        # Accumulate train_loss
        train_loss += loss.item()

        # Perform weight update after a certain number of batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()  # Update model weights
            optimizer.zero_grad()  # Clear accumulated gradients
            torch.cuda.empty_cache()  # Clear unused memory

    # Calculate the average train_loss over all batches in the epoch
    avg_train_loss = train_loss / len(train_dataloader)

    # Validation loop
    model.eval()  # Switch to evaluation mode
    valid_loss = 0
    with torch.no_grad():  # Disable gradients for validation
        for lr_images, hr_images in tqdm(valid_dataloader, desc='Validation'):
            if torch.cuda.is_available():
                lr_images = lr_images.cuda()
                hr_images = hr_images.cuda()

            outputs = model(lr_images)
            loss = criterion(outputs, hr_images)
            valid_loss += loss.item()

    # Calculate the average valid_loss over all batches in the epoch
    avg_valid_loss = valid_loss / len(valid_dataloader)

    # Print epoch statistics
    print(f"Epoch {epoch + 1}/{n_epochs} - Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_valid_loss:.4f}")

    # Free up memory after epoch
    torch.cuda.empty_cache()

# Save the trained model
torch.save(model.state_dict(), 'denoising_unet_model.pth')

Validation: 100%|██████████| 36/36 [00:33<00:00,  1.07it/s]


Epoch 1/100 - Training Loss: 3328493.5122, Validation Loss: 2517235.5833


Validation: 100%|██████████| 36/36 [00:33<00:00,  1.08it/s]


Epoch 2/100 - Training Loss: 3323517.8038, Validation Loss: 2515460.4722


Validation: 100%|██████████| 36/36 [00:33<00:00,  1.09it/s]


...

Epoch 98/100 - Training Loss: 793703.9778, Validation Loss: 489133.3997


Validation: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s]


Epoch 99/100 - Training Loss: 765524.5039, Validation Loss: 497384.5480


Validation: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s]

Epoch 100/100 - Training Loss: 774718.5665, Validation Loss: 507749.1901
